In [1]:
!wget https://raw.githubusercontent.com/happylolonly/eventsfree-ml/master/notebooks/tags/data/data.csv

--2018-12-23 14:03:13--  https://raw.githubusercontent.com/happylolonly/eventsfree-ml/master/notebooks/tags/data/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 910947 (890K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>] 889.60K  2.93MB/s    in 0.3s    

2018-12-23 14:03:14 (2.93 MB/s) - ‘data.csv’ saved [910947/910947]



In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz

In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz

In [ ]:
# !gunzip cc.ru.300.bin.gz
# !gunzip cc.ru.300.vec.gz

In [3]:
import pandas as pd

In [19]:
df = pd.read_csv('./data.csv')

df['tags'] = df['tags'].map(eval) 
df['tags'] = df['tags'].map(lambda x: set(x))
tags_white_list = {'выставка', 'праздник', 'фитнес', 'тренировка', 'trx', 'обучение', 'лекция', 'english', 'музыка', 'джаз', 'танцы', 'митап', 'дизайн', 'it', 'семинар', 'концерт', 'конкурс', 'вечеринка', 'ярмарка', 'турнир', 'тенис', 'театр', 'фотография', 'фильм', 'кино', 'дети', 'гитара', 'стартап', 'инвестиции', 'маркет', 'программирование', 'нетворкинг'}
df['tags'] = df['tags'].map(lambda x: set(x).intersection(tags_white_list))
# .intersection(tags_white_list)
df = df[df.tags.map(len) > 0]

df.head()



,title,text,tags
0,Письма из Венеции,<p>&#x41D;&#x438;&#x43A;&#x442;&#x43E; &#x43D;...,{выставка}
1,🎄🎄🎄А Вы уже ощущаете праздник и новогоднее на...,🎄🎄🎄А Вы уже ощущаете праздник и новогоднее нас...,"{фитнес, праздник}"
2,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ ...,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ э...,"{тренировка, фитнес}"
3,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \...,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \n...,"{тренировка, фитнес}"
6,Внимание!!!📌\n\nОткрытое занятие по TRX и SUP...,Внимание!!!📌\n\nОткрытое занятие по TRX и SUPE...,"{trx, тренировка, фитнес}"


In [20]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Я0-9a-zA-Z]", " ")
    df[text_field] = df[text_field].str.replace(r"\s+", " ")
    df[text_field] = df[text_field].str.lower()
    return df

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/happylol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df = standardize_text(df, 'text')
df = standardize_text(df, 'title')

df.head()

,title,text,tags
0,письма из венеции,никто не может остаться равнодушным к венеции ...,{выставка}
1,а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...,"{фитнес, праздник}"
2,poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...,"{тренировка, фитнес}"
3,бесплатная открытая тренировка по crossfit n ...,бесплатная открытая тренировка по crossfit n n...,"{тренировка, фитнес}"
6,внимание n nоткрытое занятие по trx и super 8...,внимание n nоткрытое занятие по trx и super 8 ...,"{trx, тренировка, фитнес}"


In [22]:
from collections import Counter

counter = Counter()
for tags in df['tags'].values:
    for t in tags:
        counter[t] += 1

In [23]:
counter

Counter({'выставка': 7,
         'фитнес': 17,
         'праздник': 14,
         'тренировка': 16,
         'trx': 3,
         'обучение': 11,
         'лекция': 8,
         'english': 5,
         'музыка': 36,
         'джаз': 5,
         'танцы': 4,
         'митап': 25,
         'дизайн': 3,
         'it': 7,
         'семинар': 3,
         'концерт': 3,
         'конкурс': 3,
         'вечеринка': 20,
         'ярмарка': 16,
         'турнир': 4,
         'тенис': 4,
         'театр': 6,
         'фотография': 3,
         'фильм': 4,
         'кино': 4,
         'дети': 15,
         'гитара': 3,
         'стартап': 5,
         'инвестиции': 4,
         'маркет': 4,
         'программирование': 3,
         'нетворкинг': 4})

In [24]:
arr = []
for i in counter:
    if counter[i] > 2:
        arr.append(i);
        
print(arr);

['выставка', 'фитнес', 'праздник', 'тренировка', 'trx', 'обучение', 'лекция', 'english', 'музыка', 'джаз', 'танцы', 'митап', 'дизайн', 'it', 'семинар', 'концерт', 'конкурс', 'вечеринка', 'ярмарка', 'турнир', 'тенис', 'театр', 'фотография', 'фильм', 'кино', 'дети', 'гитара', 'стартап', 'инвестиции', 'маркет', 'программирование', 'нетворкинг']


In [25]:
rows = []

for row in df.to_dict(orient='record'):
    for tag in row['tags']:
        row_copy = row.copy()
        row_copy['tag'] = tag
        
        rows.append(row_copy)
        
df = pd.DataFrame(rows)
df.head()

,tag,tags,text,title
0,выставка,{выставка},никто не может остаться равнодушным к венеции ...,письма из венеции
1,фитнес,"{фитнес, праздник}",а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...
2,праздник,"{фитнес, праздник}",а вы уже ощущаете праздник и новогоднее настр...,а вы уже ощущаете праздник и новогоднее настр...
3,тренировка,"{тренировка, фитнес}",poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...
4,фитнес,"{тренировка, фитнес}",poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...


In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, stratify=df['tag'].values)

print(np.unique(train['tag'].values, return_counts=True))
print(np.unique(test['tag'].values, return_counts=True))

(array(['english', 'it', 'trx', 'вечеринка', 'выставка', 'гитара', 'дети',
       'джаз', 'дизайн', 'инвестиции', 'кино', 'конкурс', 'концерт',
       'лекция', 'маркет', 'митап', 'музыка', 'нетворкинг', 'обучение',
       'праздник', 'программирование', 'семинар', 'стартап', 'танцы',
       'театр', 'тенис', 'тренировка', 'турнир', 'фильм', 'фитнес',
       'фотография', 'ярмарка'], dtype=object), array([ 4,  5,  2, 14,  5,  2, 10,  3,  2,  3,  3,  2,  2,  6,  3, 17, 25,
        3,  8, 10,  2,  2,  3,  3,  4,  3, 11,  3,  3, 12,  2, 11]))
(array(['english', 'it', 'trx', 'вечеринка', 'выставка', 'гитара', 'дети',
       'джаз', 'дизайн', 'инвестиции', 'кино', 'конкурс', 'концерт',
       'лекция', 'маркет', 'митап', 'музыка', 'нетворкинг', 'обучение',
       'праздник', 'программирование', 'семинар', 'стартап', 'танцы',
       'театр', 'тенис', 'тренировка', 'турнир', 'фильм', 'фитнес',
       'фотография', 'ярмарка'], dtype=object), array([ 1,  2,  1,  6,  2,  1,  5,  2,  1,  1,  1,  

In [27]:
# %%bash

# pip install -U -q git+https://github.com/facebookresearch/fastText.git

import fastText

FIELD = 'text'

In [28]:
def save(df, path):
    with open(path, 'w+') as f:
        for _, row in df.iterrows():
            f.write('__label__{} {}\n'.format(row['tag'], row[FIELD]))

In [29]:
# save(df, '/tmp/train.data')
save(train, '/tmp/train.data')

In [34]:
model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors='/tmp/ml/fasttext/cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)

In [37]:
labels, probs = model.predict(test[FIELD].tolist())
labels = [ll[0].replace('__label__', '') for ll in labels]

In [43]:
from sklearn.metrics import classification_report
print(classification_report(test['tag'].values, labels))

In [15]:
model.quantize(qnorm=True, cutoff=100000)
model.save_model('ft_quantized.model')